In [1]:
"""
Difference-in-Differences (DiD) Analysis for Weekly Active Minutes when Implementing a New Feature  
---  

🔍 **Situation**:  
A new AI-powered search feature was introduced on the platform, and the company wanted to measure its impact on user engagement. Specifically, we examined whether the feature increased **weekly active minutes per user**.  

📌 **Task**:  
Use a **Difference-in-Differences (DiD)** approach to estimate the **causal impact** of the feature rollout. We compared **two groups**:  
- **Treatment Group**: Users who received the feature.  
- **Control Group**: Users who did not receive the feature.  
By analyzing engagement **before and after** the feature release, we aimed to isolate the true effect of the feature from general trends.  

✨ **Action**:  
1. Created a **synthetic dataset** mimicking real-world engagement data.  
2. Defined key variables:  
   - **Treatment Group vs. Control Group**  
   - **Pre-treatment vs. Post-treatment Periods**  
3. Ran a Bayesian **Difference-in-Differences model** using `causalpy`.  
4. Estimated the causal effect with a **posterior distribution** to quantify uncertainty.  

📈 **Result**:  
- The feature **caused a +9.7 minute increase in weekly active minutes per user**.  
- **High confidence**: The **94% Highest Density Interval (HDI) was [8.6, 11]**, meaning the effect is statistically robust.  
- **No pre-existing differences**: The control and treatment groups had similar engagement before the feature launch, supporting the **parallel trends assumption**.  
- This suggests that the feature successfully improved user engagement.  

🚀 **Next Steps / Additional Analysis**  
- **Explore user segments**: Does the effect vary by user demographics or past engagement?  
- **Test long-term retention impact**: Does engagement stay elevated over time?  
- **Optimize feature rollout**: Consider further A/B testing with variations of the AI-powered search.  
- **Monetization Strategy**: If engagement is higher, could this translate into revenue gains?  

✍ **Author**: Justin Wall  
📅 **Updated**: 03/22/2025  
"""

'\nDifference-in-Differences (DiD) Analysis for Weekly Active Minutes when Implementing a New Feature  \n---  \n\n🔍 **Situation**:  \nA new AI-powered search feature was introduced on the platform, and the company wanted to measure its impact on user engagement. Specifically, we examined whether the feature increased **weekly active minutes per user**.  \n\n📌 **Task**:  \nUse a **Difference-in-Differences (DiD)** approach to estimate the **causal impact** of the feature rollout. We compared **two groups**:  \n- **Treatment Group**: Users who received the feature.  \n- **Control Group**: Users who did not receive the feature.  \nBy analyzing engagement **before and after** the feature release, we aimed to isolate the true effect of the feature from general trends.  \n\n✨ **Action**:  \n1. Created a **synthetic dataset** mimicking real-world engagement data.  \n2. Defined key variables:  \n   - **Treatment Group vs. Control Group**  \n   - **Pre-treatment vs. Post-treatment Periods**  \n

In [6]:
# General Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# EconML Libraries
from econml.metalearners import SLearner, TLearner, XLearner
from sklift.datasets import fetch_criteo

# Scikit-learn Libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Display settings
%matplotlib inline
sns.set(style="whitegrid")

In [10]:
import numpy as np
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Number of samples
n_samples = 10_000

# Generate customer demographic & behavioral features
age = np.random.randint(18, 70, n_samples)  # Age of customer
income = np.random.normal(60000, 15000, n_samples).astype(int)  # Income level
past_purchases = np.random.poisson(3, n_samples)  # Number of past purchases
engagement_score = np.random.uniform(0, 1, n_samples)  # Online engagement level (0-1)
loyalty_tier = np.random.choice(["Bronze", "Silver", "Gold", "Platinum"], n_samples, p=[0.4, 0.3, 0.2, 0.1])  

# Encode loyalty tier as ordinal values
loyalty_map = {"Bronze": 0, "Silver": 1, "Gold": 2, "Platinum": 3}
loyalty_tier_encoded = np.array([loyalty_map[tier] for tier in loyalty_tier])

# Assign treatment randomly (50% chance of getting the promotion)
treatment = np.random.binomial(1, 0.5, n_samples)  

# Define the base purchase probability (before any treatment)
base_purchase_prob = 0.2 + (0.01 * past_purchases) + (0.05 * loyalty_tier_encoded) + (0.03 * engagement_score)

# Define uplift effect for treatment group
uplift = (treatment * (0.05 + 0.02 * loyalty_tier_encoded + 0.01 * engagement_score))  

# Compute final purchase probability
final_purchase_prob = np.clip(base_purchase_prob + uplift, 0, 1)

# Generate the actual purchase outcome based on the probability
purchase = np.random.binomial(1, final_purchase_prob)

# Create DataFrame
df = pd.DataFrame({
    "age": age,
    "income": income,
    "past_purchases": past_purchases,
    "engagement_score": engagement_score,
    "loyalty_tier": loyalty_tier,
    "treatment": treatment,
    "purchase": purchase
})

# Display sample
df.head()

,age,income,past_purchases,engagement_score,loyalty_tier,treatment,purchase
0,56,47214,7,0.893016,Gold,0,0
1,69,67130,2,0.692129,Silver,1,0
2,46,69486,3,0.528525,Silver,1,0
3,32,52887,3,0.965471,Gold,0,0
4,60,48423,2,0.849591,Platinum,0,0


In [ ]:
# Replace 'feature_cols', 'treatment_col', and 'outcome_col' with actual column names
feature_cols = ['feature1', 'feature2', 'feature3']  # Example features
treatment_col = 'treatment'  # Binary treatment indicator
outcome_col = 'outcome'  # Binary or continuous outcome

# Split data into features, treatment, and outcome
X = data[feature_cols]
T = data[treatment_col]
Y = data[outcome_col]

# Split into training and test sets
X_train, X_test, T_train, T_test, Y_train, Y_test = train_test_split(X, T, Y, test_size=0.3, random_state=42)
